In [2]:
import os
import numpy as np
from pixmappy import DESMaps, Gnomonic

In [3]:
%env CAL_PATH=/u/ec/roodman/kipacdisk/Pixmappy/Y6A1_ASTROMETRY

env: CAL_PATH=/u/ec/roodman/kipacdisk/Pixmappy/Y6A1_ASTROMETRY


In [4]:
# Show that we are using CAL_PATH to find files
try:
    print('Reading files on path', os.environ['CAL_PATH'])
except KeyError:
    print('Set CAL_PATH environment variable to include directories with astrometric solutions')

# Read solution set from default files.
# Optional argument inhibits use/creation of python pickle files.
maps = DESMaps()

# Acquire the WCS for desired exposure, CCD combination
# ValueError will be raised if there is no astrometric solution for this combination.
expnum = 228724
ccdnum = 'N8'
wcs = maps.getDESWCS(expnum, ccdnum)

# Map a single pair of pixel coordinates to RA, Dec, specifying object color
x = 1088.4
y = 3115.2
c = 1.4
ra,dec = wcs.toSky(x,y,c)

print('x/y',x,y,'map to RA/Dec:',ra,dec)

# We can also request a WCS by the CCDNUM of the device
wcs2 = maps.getDESWCS(expnum, ccdnum)
print('With CCDNUM:')
ra,dec = wcs2.toSky(x,y,c)
print('x/y',x,y,'map to RA/Dec:',ra,dec)

# Map these back
print('Inversion yields', wcs.toPix(ra,dec,c))

# Map a set of coordinates with varying colors.
# c could be a scalar and would be broadcast
x = np.arange(900.,1500.,100.)
y = x + 1000.
c = np.arange(0.9,1.5,0.1)
ra,dec = wcs.toSky(x,y,c)
print('Array outputs:')
print('RA:',ra)
print('Dec:',dec)

# Now we're going to ask the WCS to reproject coordinates
# into a gnomonic projection about a new point, in
# units of degrees

ra0 = 37.
dec0 = -25.
wcs.reprojectTo( Gnomonic(ra0,dec0) )

# And we just use the wcs as a function object to map pixel
# coordinates into projection-plane coordinates:
x = 555.
y = 4321.
c = 0.5
print('xi,eta are', wcs(x,y,c))

# We can get the local Jacobian of the map from pixel
# to project coordinates
print('Jacobian:',wcs.jacobian(x,y,c))

# Now get the estimated covariance matrix of astrometric
# errors for this exposure.
cov = maps.getCovariance(expnum)
print('Covariance matrix:',cov)
# Is there a warning about quality of this covariance?
print('Warning for covariance?',maps.covarianceWarning(expnum))

Reading files on path /u/ec/roodman/kipacdisk/Pixmappy/Y6A1_ASTROMETRY
x/y 1088.4 3115.2 map to RA/Dec: 329.23619277478275 -54.647148036139484
With CCDNUM:
x/y 1088.4 3115.2 map to RA/Dec: 329.23619277478275 -54.647148036139484
Inversion yields (1088.3999965245252, 3115.200005621793)
Array outputs:
RA: [329.08367863 329.09599984 329.10832593 329.1206583  329.13299008
 329.14533281]
Dec: [-54.63143919 -54.63891266 -54.64638169 -54.6538524  -54.66132458
 -54.66879232]
xi,eta are (-56.351613713986275, -67.94191311940911)
Jacobian: [[ 9.39693563e-05  1.62840374e-04]
 [-9.62616441e-05  1.82642804e-04]]
Covariance matrix: [[ 2.50000000e+01 -7.43337882e-17]
 [-7.43337882e-17  2.50000000e+01]]
Warning for covariance? False


In [7]:
# Plot the pixel size across the DECam focal plane
maps = DESMaps()

# Acquire the WCS for desired exposure, CCD combination
# ValueError will be raised if there is no astrometric solution for this combination.
expnum = 228724
ccdnum = 'N8'
wcs = maps.getDESWCS(expnum, ccdnum)

# Map a single pair of pixel coordinates to RA, Dec, specifying object color
x = 1088.4
y = 3115.2
c = 1.4
ra,dec = wcs.toSky(x,y,c)

print('x/y',x,y,'map to RA/Dec:',ra,dec)
print('Jacobian:',wcs.jacobian(x,y,c))
print('sqrt(Area)',np.sqrt(np.abs(np.linalg.det(wcs.jacobian(x,y,c)))*(3600.*3600.)))

# do several?
xs = np.array([800.1,900.])
ys = np.array([2000.4,2300.1])
print('array of sqrt(Area)',np.sqrt(np.abs(np.linalg.det(wcs.jacobian(xs,ys,c)))*(3600.*3600.)))



# Now we're going to ask the WCS to reproject coordinates
# into a gnomonic projection about a new point, in
# units of degrees

ra0 = 37.
dec0 = -25.
wcs.reprojectTo( Gnomonic(ra,dec) )

# And we just use the wcs as a function object to map pixel
# coordinates into projection-plane coordinates:
#x = 555.
#y = 4321.
#c = 0.5
print('xi,eta are', wcs(x,y,c))
print('xi,eta are', wcs(x+1.,y,c))
print('xi,eta are', wcs(x-1.,y,c))
print('xi,eta are', wcs(x,y+1.,c))
print('xi,eta are', wcs(x,y-1.,c))



# We can get the local Jacobian of the map from pixel
# to project coordinates
print('Jacobian:',wcs.jacobian(x,y,c))
print('Area',np.abs(np.linalg.det(wcs.jacobian(x,y,c)))*(3600.*3600.))
print('sqrt(Area)',np.sqrt(np.abs(np.linalg.det(wcs.jacobian(x,y,c)))*(3600.*3600.)))




x/y 1088.4 3115.2 map to RA/Dec: 329.23619277478275 -54.647148036139484
Jacobian: [[-2.69465787e-07  7.30182400e-05]
 [-7.32339338e-05 -1.32237023e-07]]
sqrt(Area) 0.26325450364137776
array of sqrt(Area) [0.26281378 0.26248864]
xi,eta are (-0.0, 0.0)
xi,eta are (-1.523922135028085e-06, -7.320948563990825e-05)
xi,eta are (1.5285967869964236e-06, 7.321382827703424e-05)
xi,eta are (7.299647623772779e-05, -1.386574543270811e-06)
xi,eta are (-7.299065306604561e-05, 1.3917860284219083e-06)
Jacobian: [[-1.52625946e-06  7.29935647e-05]
 [-7.32116570e-05 -1.38918029e-06]]
Area 0.06928545684271423
sqrt(Area) 0.26322130772928365


In [6]:
help(wcs)

Help on WCS in module pixmappy.PixelMapCollection object:

class WCS(PixelMap)
 |  WCS(name, pmap, projection, scale=1.0)
 |  
 |  A WCS is a PixelMap that is augmented by a projection from the world coordinates
 |  to definitive sky position (ra/dec). The method toSky() will return RA, Dec arrays
 |  corresponding to the input pixel coordinates.
 |  Also can alternatively supply a second projection such that __call__ method 
 |  returns the coordinates in the second projection's system.
 |  A projection is anything that has toSky() and toPix() methods going to/from
 |  RA, Dec.  The projections `ICRS` and `Gnomonic` are defined in this file.
 |  The `scale` factor is multiplied into world coordinates to turn them into degrees.
 |  
 |  Method resolution order:
 |      WCS
 |      PixelMap
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __call__(self, x, y, c=None)
 |      Map the input pixel coordinates to the world coordinates of the original
 |      `PixelMap`, or to